## Monitor transactions using anomaly detection model.     
---
**NOTE**: 

In real life scenarios financial transaction are dynamically evolving graphs. Performing anomaly detection inference on graph embeddings in live Transaction Monitoring Systems will require to update the graph and node representations after new transactions arrive. Recomputing entire graph for every newly arrived transaction will lead to unaxeptable delayes and even monitoring system failure. This problem  will be more sever if large amount of updates happen in a short time window.

Contact us at Logical Clocks and we will help you to setup end to end graph based deep anomaly detection for live Transaction Monitoring Systems. 

---

In [1]:
import json
import numpy as np

![Image7-Monitor.png](./images/model_registry.gif)

# Query Model Repository for best anomaly detection model

In [2]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [3]:
MODEL_NAME="ganAml"
EVALUATION_METRIC="loss"

In [4]:
best_model = mr.get_best_model(MODEL_NAME, EVALUATION_METRIC, "min")

In [5]:
print('Model name: ' + best_model.name)
print('Model version: ' + str(best_model.version))
print(best_model.training_metrics)

Model name: ganAml
Model version: 2
{'loss': '-0.03453196585178375'}


### Create the deployment
Here, we fetch the model we want from the model registry and define a configuration for the deployment. For the configuration, we need to specify the serving type (default or KFserving) and in this case, since we use default serving and an sklearn model, we need to give the location of the prediction script.

In [6]:
import hsml

conn = hsml.connection()
mr = conn.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.


In [7]:
# Use the model name from notebook 4, where we registered the model
model = mr.get_model(best_model.name, version=best_model.version)

In [8]:
# Create serving instance
SERVING_NAME = "ganTfServing"

# Give it any name you want
model.deploy(name=SERVING_NAME, model_server="TENSORFLOW_SERVING", serving_tool="DEFAULT")

Deployment started, explore it at https://hopsworks0.logicalclocks.com/p/119/deployments/2


Deployment(name: 'ganTfServing')

## Fetch model server object 

In [9]:
ms = conn.get_model_serving()
deployment = ms.get_deployment('ganTfServing')

### Check Model Serving for active deployments
![Image7-Monitor.png](./images/deployment.gif)

In [10]:
# Get serving status
deployment.get_state()

PredictorState(status: 'Stopped')

In [11]:
# Start deployment
deployment.start()

  0%|          | 0/1 [00:00<?, ?it/s]

##### Retrieve serving vectors and send prediction requests to the served model using Hopsworks REST API

In [12]:
import hsfs
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
feature_view_non_sar = fs.get_feature_view('non_sar_transactions_view', 1)
feature_view_sar = fs.get_feature_view('sar_transactions_view', 1)

#### Retrieve test feature vector

In [14]:
feature_view_non_sar.get_feature_vector({'id': "0016359b"})

[1,
 0.13333333333333333,
 0.10375241959933768,
 0.41951605906870604,
 0.5523306125312814,
 0.3333333333333333,
 0.5159525808594092,
 0.7698600424453773,
 0.23520631848583132,
 [0.9999812245368958,
  0.9999898076057434,
  0.999980628490448,
  0.9999653100967407,
  0.9999864101409912,
  0.9999986290931702,
  1.0,
  0.9999991059303284,
  0.9999988675117493,
  0.9999955892562866,
  0.9999983310699463,
  0.9999964833259583,
  1.0,
  0.9999929666519165,
  0.9999990463256836,
  1.0,
  1.0,
  0.9999840259552002,
  0.999994158744812,
  0.9999998807907104,
  0.9999914765357971,
  1.0,
  1.0,
  0.9999879002571106,
  0.9999469518661499,
  0.9999988079071045,
  0.999983549118042,
  0.9999848008155823,
  0.9999995231628418,
  0.9999985098838806,
  0.9999852776527405,
  0.9999995231628418]]

In [15]:
ids_to_score = ["0016359b", 
                "001dcc27", 
                "0054a022", 
                "00d6b609", 
                "00e14860", 
                "00e39a1b", 
                "014ed5cb", 
                "01ce3306", 
                "01fa19ae", 
                "01fa1d01", 
                "036dce03", 
                "03e09be4", 
                "04b23f4b"]

In [20]:
def flat2gen(alist):
    for item in alist:
        if isinstance(item, list):
            for subitem in item: yield subitem
        else:
            yield item

def model_server(input):
    data = {"inputs": [input]}
    return deployment.predict(data)

for node_id in ids_to_score:
    serving_vector = np.array(list(flat2gen(feature_view_non_sar.get_feature_vector({'id': node_id})))).reshape(1,41).tolist()
    print(" anomaly score for node_id ", node_id, " : ",   model_server(serving_vector)["outputs"])

 anomaly score for node_id  0016359b  :  0.790985763
 anomaly score for node_id  001dcc27  :  0.760188341
 anomaly score for node_id  0054a022  :  0.778505445
 anomaly score for node_id  00d6b609  :  0.78152281
 anomaly score for node_id  00e14860  :  0.795529366
 anomaly score for node_id  00e39a1b  :  0.753337741
 anomaly score for node_id  014ed5cb  :  0.760115087
 anomaly score for node_id  01ce3306  :  0.757739365
 anomaly score for node_id  01fa19ae  :  0.766529322
 anomaly score for node_id  01fa1d01  :  0.768219531
 anomaly score for node_id  036dce03  :  0.766904116
 anomaly score for node_id  03e09be4  :  0.76641053
 anomaly score for node_id  04b23f4b  :  0.777334511
